
## Binomial Options Pricing Model
This python notebook covers the implementation of the Binomial Options Pricing Model. If you are interested in learning about what this is, then check out this article explaining it: https://bathquantgroup.netlify.app/blog/bopm

The function below takes the parameters for the initial stock price (S), strike price (K), time to maturity (T), risk-free rate (r), volatility (sigma), number of steps in the binomial tree (N), and the type of the option (either 'call' or 'put'). 


- Up and Down Factors: Calculates the up (u) and down (d) factors based on the volatility and time step. These factors determine how much the stock price can rise or fall in each step.
- Risk-Neutral Probability q: This is the probability of the stock price moving up in the risk-neutral world, calculated using the risk-free rate, and the up and down factors.
- Asset Prices at Maturity: Computes the possible prices of the underlying asset at the expiration of the option.
- Option Values at Maturity: Depending on whether it's a call or put option, this computes the payoff for each possible asset price at maturity.
- Backward Iteration through the Tree: Using dynamic programming, the script calculates the option value at each preceding node by discounting the expected payoff from the next step.
- Output: Finally, the price of the option today is given by the first element of the option values array after completing the backward iteration.

This script can be modified to accommodate American options or to change the assumptions about dividends or other factors influencing the underlying asset's price.

In [10]:
import numpy as np

def binomial_option_pricing(S, K, T, r, sigma, N, option_type='call'):
    """
    Parameters:
    S : float - initial stock price
    K : float - strike price of the option
    T : float - time to expiration in years
    r : float - risk-free interest rate (annual)
    sigma : float - volatility of the underlying stock
    N : int - number of steps in the binomial tree
    option_type : str - type of option ('call' or 'put')
    
    Returns:
    float - estimated option price
    """
    # Time interval
    dt = T / N
    # Calculate up and down factors
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    
    # Risk-neutral probability
    q = (np.exp(r * dt) - d) / (u - d)
    
    # Initialize asset prices at maturity
    asset_prices = np.array([S * d**j * u**(N - j) for j in range(N + 1)])
    
    # Initialize option values at maturity
    if option_type == 'call':
        option_values = np.maximum(asset_prices - K, 0)
    elif option_type == 'put':
        option_values = np.maximum(K - asset_prices, 0)
    
    # Iterate backwards through the tree
    for i in range(N - 1, -1, -1):
        option_values = (q * option_values[:-1] + (1 - q) * option_values[1:]) * np.exp(-r * dt)
    
    return option_values[0]


## Examples
To effectively test `binomial_option_pricing()`, we should consider a variety of scenarios that could affect the price of options. These scenarios should include variations in all input parameters like the initial stock price, strike price, time to expiration, risk-free rate, volatility, number of steps, and option types. Below are a few example test cases:


### Test Case 1: Basic Functionality Test
Objective: Test the function with basic, reasonable parameters to ensure it handles straightforward inputs correctly.
Inputs:
- Initial Stock Price (S): $100
- Strike Price (K): $100
- Time to Expiration (T): 1 year
- Risk-Free Rate (r): 5% (0.05)
- Volatility (sigma): 20% (0.2)
- Number of Steps (N): 50
- Option Type: 'call'

Expected Output: Roughly around the Black-Scholes price for these parameters, approximately $10.45

In [2]:
S = 100  # initial stock price
K = 100  # strike price
T = 1    # time to expiration
r = 0.05 # risk-free rate
sigma = 0.2  # volatility
N = 50   # number of steps

# Calculate call and put prices
call_price = binomial_option_pricing(S, K, T, r, sigma, N, 'call')
put_price = binomial_option_pricing(S, K, T, r, sigma, N, 'put')

print(f"Call Option Price: {call_price:.2f}")
print(f"Put Option Price: {put_price:.2f}")

Call Option Price: 10.41
Put Option Price: 5.53


### Test Case 2: Deep In-The-Money Call Option
Objective: Evaluate a call option that is deep in-the-money (current stock price far above strike price).

Expected Output: Significantly higher than the basic test case, reflecting the greater intrinsic value.

In [3]:
S = 150  # initial stock price
K = 100  # strike price
T = 1    # time to expiration
r = 0.05 # risk-free rate
sigma = 0.2  # volatility
N = 50   # number of steps

# Calculate call and put prices
call_price = binomial_option_pricing(S, K, T, r, sigma, N, 'call')

print(f"Call Option Price: {call_price:.2f}")

Call Option Price: 54.97


### Test Case 3: Out-of-The-Money Put Option
Objective: Assess a put option that is out-of-the-money (current stock price above the strike price).

Expected Output: Should have some value reflecting the time value despite being out of the money, but much less than an in-the-money option.


In [4]:
S = 90  # initial stock price
K = 100  # strike price
T = 1    # time to expiration
r = 0.05 # risk-free rate
sigma = 0.2  # volatility
N = 50   # number of steps

# Calculate put price
put_price = binomial_option_pricing(S, K, T, r, sigma, N, 'put')

print(f"Put Option Price: {put_price:.2f}")

Put Option Price: 10.20


### Test Case 5: Short Time to Expiration
Objective: Verify how a shorter time to expiration affects the pricing, particularly near expiry.

Expected Output: Much lower than for longer periods due to less time for the stock to move.


In [5]:
S = 100  # initial stock price
K = 100  # strike price
T = 0.1    # time to expiration
r = 0.05 # risk-free rate
sigma = 0.2  # volatility
N = 10   # number of steps (fewer steps due to shorter period)

# Calculate call price
call_price = binomial_option_pricing(S, K, T, r, sigma, N, 'call')

print(f"Call Option Price: {call_price:.2f}")

Call Option Price: 2.71
